# Tinkoff Investments Multi-Stock Alpha Strategy

This notebook implements and backtests an alpha strategy across multiple Russian stocks.

In [1]:
import os
from dotenv import load_dotenv
from tinkoff.invest import Client
import pandas as pd
import numpy as np

# Load environment variables
load_dotenv()

# Get token from environment variables
TOKEN = os.getenv('TINKOFF_TOKEN')

if not TOKEN:
    raise ValueError("Please set TINKOFF_TOKEN in your .env file")

In [2]:
from tinkoff.invest import CandleInterval
def get_stock_data(figi, from_date, to_date, interval=CandleInterval.CANDLE_INTERVAL_DAY):
    with Client(TOKEN) as client:
        candles = client.market_data.get_candles(
            figi=figi,
            from_=from_date,
            to=to_date,
            interval=interval
        )
        
        df = pd.DataFrame([{
            'time': c.time,
            'open': c.open.units + c.open.nano / 1e9,
            'high': c.high.units + c.high.nano / 1e9,
            'low': c.low.units + c.low.nano / 1e9,
            'close': c.close.units + c.close.nano / 1e9,
            'volume': c.volume
        } for c in candles.candles])
        
        return df

In [3]:
def calculate_alpha1_multiple(stock_data):
    alpha_signals = {}
    
    for stock_name, df in stock_data.items():
        returns = df['close'].pct_change()
        returns_stddev = returns.rolling(window=20).std()
        
        power_term = np.where(returns < 0, 
                            returns_stddev, 
                            df['close'])
        signed_power = np.sign(power_term) * (np.abs(power_term) ** 2)
        
        ts_argmax = pd.Series(signed_power).rolling(5).apply(np.argmax)
        alpha = ts_argmax.rank(pct=True) - 0.5
        
        alpha_signals[stock_name] = alpha
    
    return pd.DataFrame(alpha_signals)


def calculate_alpha2_multiple(stock_data):
    alpha_signals = {}
    
    for stock_name, df in stock_data.items():
        # Calculate volume changes
        log_volume = np.log(df['volume'])
        volume_delta = log_volume.diff(2)
        volume_delta_rank = volume_delta.rank(pct=True)
        
        # Calculate returns
        returns = (df['close'] - df['open']) / df['open']
        returns_rank = returns.rank(pct=True)
        
        # Calculate correlation
        correlation = volume_delta_rank.rolling(window=6).corr(returns_rank)
        
        # Final alpha signal
        alpha = -1 * correlation
        
        alpha_signals[stock_name] = alpha
    
    return pd.DataFrame(alpha_signals)


In [4]:
def neutralize_weights(weights):
    # Demean to make sum = 0
    weights = weights.sub(weights.mean(axis=1), axis=0)
    
    # Scale so absolute values sum to 1
    abs_sum = weights.abs().sum(axis=1)
    weights = weights.div(abs_sum, axis=0)
    
    return weights

In [5]:
# Implement Alpha3 strategy
def calculate_alpha3_multiple(stock_data, k=4):
    """Calculate Alpha3 signals for multiple stocks"""
    # Extract required data
    closes = pd.DataFrame({
        name: data['close'] 
        for name, data in stock_data.items()
    })
    
    volumes = pd.DataFrame({
        name: data['volume']
        for name, data in stock_data.items() 
    })
    
    highs = pd.DataFrame({
        name: data['high']
        for name, data in stock_data.items()
    })
    
    lows = pd.DataFrame({
        name: data['low'] 
        for name, data in stock_data.items()
    })

    # Calculate VWAP
    vwap = (closes.shift(1) * volumes.shift(1)).rolling(k).sum() / volumes.shift(1).rolling(k).sum()
    
    # Calculate weights
    weights = (highs.shift(1) * lows.shift(1))**0.5 - vwap
    
    return weights

In [9]:
from tinkoff.invest import InstrumentStatus
from tinkoff.invest.schemas import InstrumentExchangeType

def get_all_shares():
    with Client(TOKEN) as client:
        return client.instruments.shares(instrument_status=InstrumentStatus.INSTRUMENT_STATUS_BASE, instrument_exchange=InstrumentExchangeType.INSTRUMENT_EXCHANGE_UNSPECIFIED)

instruments = get_all_shares().instruments
instruments

[Share(figi='BBG000CKVSG8', ticker='CNX', class_code='SPBXM', isin='US12653C1080', lot=1, currency='usd', klong=Quotation(units=0, nano=0), kshort=Quotation(units=0, nano=0), dlong=Quotation(units=0, nano=0), dshort=Quotation(units=0, nano=0), dlong_min=Quotation(units=0, nano=0), dshort_min=Quotation(units=0, nano=0), short_enabled_flag=False, name='CNX Resources Corporation', exchange='unknown', ipo_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), issue_size=211199776, country_of_risk='US', country_of_risk_name='Соединенные Штаты Америки', sector='energy', issue_size_plan=500000000, nominal=MoneyValue(currency='usd', units=0, nano=10000000), trading_status=<SecurityTradingStatus.SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING: 1>, otc_flag=False, buy_available_flag=False, sell_available_flag=False, div_yield_flag=False, share_type=<ShareType.SHARE_TYPE_COMMON: 1>, min_price_increment=Quotation(units=0, nano=10000000), api_trade_available_flag=True, uid='82f8d

In [23]:
# Create mapping of ticker to FIGI and get data for selected stocks
ticker_to_figi = {instrument.ticker: instrument.figi for instrument in instruments}

# Define target stocks
target_stocks = ['GAZP', 'MTSS', 'SBER']

# Get historical data for each stock
from datetime import datetime, timedelta, timezone
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=365*5)

prices_data = {}
for ticker in target_stocks:
    figi = ticker_to_figi[ticker]
    data = get_stock_data(figi, start_date, end_date)
    prices_data[ticker] = data

prices_data

{'GAZP':                           time    open    high     low   close    volume
 0    2020-04-01 00:00:00+00:00  178.03  181.38  177.50  180.06   4043682
 1    2020-04-02 00:00:00+00:00  183.80  190.00  180.25  187.04   8781991
 2    2020-04-03 00:00:00+00:00  187.01  193.85  185.72  189.77   9236362
 3    2020-04-06 00:00:00+00:00  192.24  193.44  189.44  191.10   6293357
 4    2020-04-07 00:00:00+00:00  192.90  195.13  189.80  191.22   6700026
 ...                        ...     ...     ...     ...     ...       ...
 1251 2025-03-27 00:00:00+00:00  161.50  162.40  154.90  155.25   9086359
 1252 2025-03-28 00:00:00+00:00  155.00  156.11  145.20  145.43  16958041
 1253 2025-03-29 00:00:00+00:00  145.41  145.72  143.99  143.99    875183
 1254 2025-03-30 00:00:00+00:00  143.99  143.99  143.99  143.99     65338
 1255 2025-03-31 00:00:00+00:00  142.45  147.49  140.50  144.66   7349714
 
 [1256 rows x 6 columns],
 'MTSS':                           time    open    high     low   close  vol

In [30]:
# Calculate alpha1 signals
alpha_target_percent = calculate_alpha1_multiple(prices_data)
alpha_target_percent = neutralize_weights(alpha_target_percent)
alpha_target_percent[18:]

,GAZP,MTSS,SBER
18,NaN,NaN,NaN
19,NaN,NaN,NaN
20,NaN,NaN,NaN
21,NaN,NaN,NaN
22,-0.5,0.500000,NaN
...,...,...,...
1251,0.5,-0.260952,-0.239048
1252,0.5,-0.254260,-0.245740
1253,0.5,-0.250898,-0.249102
1254,0.5,-0.334770,-0.165230


In [27]:
import vectorbt as vbt
from vectorbt.portfolio.enums import SizeType

prices = pd.DataFrame({
    name: data['close']
    for name, data in prices_data.items()
})

# Create portfolio using vectorbt
portfolio = vbt.Portfolio.from_orders(
    prices,
    alpha_target_percent,
    size_type=SizeType.Percent,
    init_cash=1000000,                 # Initial capital
    fees=0.001,                        # 0.1% trading fee
    freq='1D',                         # Daily data
)

In [28]:
portfolio.stats()

/var/folders/sf/1f62hgvd3331dp_s9v7xnrb40000gn/T/ipykernel_31254/1305839121.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x107e04540>. Pass column to select a single column/group.



Start                                                 0
End                                                1255
Period                               1256 days 00:00:00
Start Value                                   1000000.0
End Value                                1276975.118301
Total Return [%]                              27.697512
Benchmark Return [%]                           7.455741
Max Gross Exposure [%]                     23984.877299
Total Fees Paid                           344880.737209
Max Drawdown [%]                              35.029189
Max Drawdown Duration                 545 days 16:00:00
Total Trades                                      428.0
Total Closed Trades                               427.0
Total Open Trades                                   1.0
Open Trade PnL                            -11426.723614
Win Rate [%]                                  43.476909
Best Trade [%]                                27.342247
Worst Trade [%]                              -19

In [29]:
portfolio.plot(group_by=True)

/Users/beldim/Documents/invest_alphas/venv/lib/python3.13/site-packages/vectorbt/generic/plots_builder.py:339: UserWarning:

Subplot 'orders' does not support grouped data

/Users/beldim/Documents/invest_alphas/venv/lib/python3.13/site-packages/vectorbt/generic/plots_builder.py:339: UserWarning:

Subplot 'trade_pnl' does not support grouped data



FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#7f7f7f'},
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'c911ac30-fcc8-418c-b084-bf27d34a3dd6',
              'x': array([   0,    1,    2, ..., 1253, 1254, 1255]),
              'xaxis': 'x',
              'y': array([1.        , 1.01941755, 1.01795562, ..., 1.06313337, 1.05100524,
                          1.07455741]),
              'yaxis': 'y'},
             {'hoverinfo': 'skip',
              'legendgroup': '1',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'd18989c9-b3b0-4ec4-82e8-842d66cd1e6b',
              'x': array([   0,    1,    2, ..., 1253, 1254, 1255]),
              'xaxis': 'x',
              'y': array([1, 1, 1, ..., 1, 1, 1]),
              'yaxis': 'y'},
             {'conn

In [16]:
for ticker in target_stocks:
    portfolio[ticker].plot().show()

In [20]:
import asyncio
import os

from tinkoff.invest import (
    AsyncClient,
    CandleInstrument,
    MarketDataRequest,
    SubscribeCandlesRequest,
    SubscriptionAction,
    SubscriptionInterval,
)


async def main():
    async def request_iterator():
        yield MarketDataRequest(
            subscribe_candles_request=SubscribeCandlesRequest(
                subscription_action=SubscriptionAction.SUBSCRIPTION_ACTION_SUBSCRIBE,
                instruments=[
                    CandleInstrument(
                        figi=ticker_to_figi['GAZP'],
                        interval=SubscriptionInterval.SUBSCRIPTION_INTERVAL_ONE_MINUTE,
                    )
                ],
            )
        )
        while True:
            await asyncio.sleep(1)

    async with AsyncClient(TOKEN) as client:
        async for marketdata in client.market_data_stream.market_data_stream(
            request_iterator()
        ):
            print(marketdata)


await main()


/Users/beldim/Documents/invest_alphas/venv/lib/python3.13/site-packages/google/protobuf/symbol_database.py:55: UserWarning:

SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.



MarketDataResponse(subscribe_candles_response=SubscribeCandlesResponse(tracking_id='65708dce7a55adbae1551182a11ceb72', candles_subscriptions=[CandleSubscription(figi='BBG004730RP0', interval=<SubscriptionInterval.SUBSCRIPTION_INTERVAL_ONE_MINUTE: 1>, subscription_status=<SubscriptionStatus.SUBSCRIPTION_STATUS_SUCCESS: 1>, instrument_uid='962e2a95-02a9-4171-abd7-aa198dbe643a', waiting_close=False, stream_id='3133a015-bd55-4953-8f6d-a8dfd9640e11', subscription_id='f90dbb8f-1a47-4513-aea5-b30c19df76f8', candle_source_type=<CandleSource.CANDLE_SOURCE_UNSPECIFIED: 0>)]), subscribe_order_book_response=None, subscribe_trades_response=None, subscribe_info_response=None, candle=None, trade=None, orderbook=None, trading_status=None, ping=None, subscribe_last_price_response=None, last_price=None)
MarketDataResponse(subscribe_candles_response=None, subscribe_order_book_response=None, subscribe_trades_response=None, subscribe_info_response=None, candle=Candle(figi='BBG004730RP0', interval=<Subscrip

CancelledError: 